# 7. 의사결정 나무 (Decision Tree)
컴퓨터 알고리즘에서 즐겨 사용하는 tree 구조를 사용하고, 각 분기점(node)에는 목표 값을 가장 잘 분류할 수 있는 분석대상의 속성(설명 변수)들이 위치한다.  
각각의 분기점에서 최적 속성을 선택 시 속성을 기준으로 분류한 값들이 구분되는 정도를 측정한다. 이때 Entropy를 주로 활용하는데, Entropy가 낮을수록 분류가 잘 된 것이고, Entropy가 일정 수준 이하로 낮아질 때까지 앞의 과정을 반복한다.  
즉, 각 분기점에서 최적의 속성을 찾기 위해 분류 정도를 평가하는 기준이 entropy이다.

In [1]:
import pandas as pd
import numpy as np

# Breast Cancer 데이터셋 가져오기 (출처: UCI ML Repository)
uci_path = 'https://archive.ics.uci.edu/ml/machine-learning-databases/\
breast-cancer-wisconsin/breast-cancer-wisconsin.data'
df= pd.read_csv(uci_path, header=None) # 불러올 데이터가 header가 없는 경우 header=None 을 넣어야함

df.columns = ['id','clump','cell_size','cell_shape','adhesion','epithlial',
              'bare_nuclei','chromatin','normal_nucleoli','mitoses','class']
print(df.head())

        id  clump  cell_size  cell_shape  adhesion  epithlial bare_nuclei  \
0  1000025      5          1           1         1          2           1   
1  1002945      5          4           4         5          7          10   
2  1015425      3          1           1         1          2           2   
3  1016277      6          8           8         1          3           4   
4  1017023      4          1           1         3          2           1   

   chromatin  normal_nucleoli  mitoses  class  
0          3                1        1      2  
1          3                2        1      2  
2          3                1        1      2  
3          3                7        1      2  
4          3                1        1      2  


In [2]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               699 non-null    int64 
 1   clump            699 non-null    int64 
 2   cell_size        699 non-null    int64 
 3   cell_shape       699 non-null    int64 
 4   adhesion         699 non-null    int64 
 5   epithlial        699 non-null    int64 
 6   bare_nuclei      699 non-null    object
 7   chromatin        699 non-null    int64 
 8   normal_nucleoli  699 non-null    int64 
 9   mitoses          699 non-null    int64 
 10  class            699 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 60.2+ KB
None


In [3]:
print(df.describe())

                 id       clump   cell_size  cell_shape    adhesion  \
count  6.990000e+02  699.000000  699.000000  699.000000  699.000000   
mean   1.071704e+06    4.417740    3.134478    3.207439    2.806867   
std    6.170957e+05    2.815741    3.051459    2.971913    2.855379   
min    6.163400e+04    1.000000    1.000000    1.000000    1.000000   
25%    8.706885e+05    2.000000    1.000000    1.000000    1.000000   
50%    1.171710e+06    4.000000    1.000000    1.000000    1.000000   
75%    1.238298e+06    6.000000    5.000000    5.000000    4.000000   
max    1.345435e+07   10.000000   10.000000   10.000000   10.000000   

        epithlial   chromatin  normal_nucleoli     mitoses       class  
count  699.000000  699.000000       699.000000  699.000000  699.000000  
mean     3.216023    3.437768         2.866953    1.589413    2.689557  
std      2.214300    2.438364         3.053634    1.715078    0.951273  
min      1.000000    1.000000         1.000000    1.000000    2.0000

위의 값을 보면 'bare_nuclei'열을 제외한 나며지 열은 모두 숫자다(int64). unique()를 이용해서 확인해 보고 replace와 dropna를 이용해서 누락행을 지워보자.

In [4]:
print(df['bare_nuclei'].unique())

['1' '10' '2' '4' '3' '9' '7' '?' '5' '8' '6']


In [5]:
df['bare_nuclei'].replace('?', np.nan, inplace = True) # ?들 대신 NaN을 넣음
df.dropna(subset=['bare_nuclei'], axis=0, inplace=True) # NaN 있는 행을 지움
df['bare_nuclei'] = df['bare_nuclei'].astype('int') # 아마 남은 행의 요소들을 int로 바꾸는 거 같음

이제 설명 변수 X와 예측 변수로 사용할 열을 분리한다. 그 후에 설명 변수를 정규화(서로 다른 데이터 값들을 모음)하고 훈련 데이터와 검증 데이터를 분리한다.

In [6]:
X = df[['clump','cell_size','cell_shape','adhesion','epithlial',
        'bare_nuclei','chromatin','normal_nucleoli','mitoses']] # 설명 변수 X
y = df['class']

from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=10)

이제 DecisionTreeClassifier()함수를 사용하여 모형 객체를 생성하자.  
각 분기점에서 최적의 속성을 찾기 위한 평가 기준으로 'entropy'값을 사용하낟. 그리고 'max_depth'으로 트리 레벨을 지정가능하다.  
아래처럼 5로 지정을 하면 5단계까지 가지를 확장할 수 있어진다. 레벨이 많아 질수록 예측은 정확해지지만, 너무 많은 트리는 훈련과정에서 train_data에만 지나치게 최적화되어 실제 데이터 예측능력이 떨어진다.  
따라서 적정한 레벨값을 찾는 것이 중요하다.

그리고 fit() 으로 모형을 학습시키고,  
predict() 매소드에 검증 데이터(x_test)를 전달하여 예측된 결과를 y_hat에 저장한다.

In [8]:
from sklearn import tree
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5) # tree를 5단계까지 확장
tree_model.fit(X_train,y_train)
y_hat = tree_model.predict(X_test)
print(y_hat[0:10])
print(y_test.values[0:10])

[4 4 4 4 4 4 2 2 4 4]
[4 4 4 4 4 4 2 2 4 4]


성능 평가를 통해 confusion Matrix를 계산해보자

In [9]:
from sklearn import metrics
tree_matrix = metrics.confusion_matrix(y_test, y_hat)
print(tree_matrix)

[[127   4]
 [  2  72]]


In [10]:
tree_report = metrics.classification_report(y_test, y_hat)
print(tree_report)

              precision    recall  f1-score   support

           2       0.98      0.97      0.98       131
           4       0.95      0.97      0.96        74

    accuracy                           0.97       205
   macro avg       0.97      0.97      0.97       205
weighted avg       0.97      0.97      0.97       205



confusion matrix를 보면  
양성을 정확히 예측한 TP = 127개  
양성을 악성으로 잘못 분류한 FP = 4개  
악성을 양성으로 잘못분류한 FN = 2개  
악성을 정확하게 예측한 TN = 2개이다.

f1-score 지표를 보면 양성종양(2) 예측 정확도가 0.98이고, 악성 종양(4) 예측 정확도는 0.96으로 예측능력에 큰 차이가 없다.  
즉, 평균적으로 0.97의 정확도를 가진다고 볼 수 있다.